# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lunca de jos,46.5667,25.9833,1.83,85,7,0.95,RO,1710621218
1,1,olovyannaya,50.9500,115.5667,-10.10,70,100,4.66,RU,1710621146
2,2,port-aux-francais,-49.3500,70.2167,6.13,88,100,12.50,TF,1710621219
3,3,ouricuri,-7.8825,-40.0817,29.14,48,100,0.80,BR,1710621220
4,4,uturoa,-16.7333,-151.4333,28.78,67,100,1.96,PF,1710621221


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
     'Lng',
     'Lat',
     geo = True,
     size = 'Humidity',
     color = "City",
     scale = 1,
     tiles= 'OSM',
     frame_width = 700,
     frame_height = 500
)
map_plot
# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
df= city_data_df.loc[(city_data_df['Max Temp']>25)&(city_data_df['Max Temp']<38)&(city_data_df['Cloudiness']<5)&(city_data_df['Wind Speed']<10)]

# Drop any rows with null values
df= df.dropna()


In [34]:
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,kedougou,12.5579,-12.1743,33.76,8,0,1.68,SN,1710621229
21,21,san luis de la loma,17.2706,-100.8939,29.58,58,0,4.43,MX,1710621236
64,64,lihue,21.9789,-159.3672,25.58,64,0,5.66,US,1710621217
93,93,san julian,21.0167,-102.1667,28.35,10,1,7.12,MX,1710621293
133,133,kavaratti,10.5669,72.6420,27.50,70,3,4.29,IN,1710621326


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
13,kedougou,SN,12.5579,-12.1743,8,
21,san luis de la loma,MX,17.2706,-100.8939,58,
64,lihue,US,21.9789,-159.3672,64,
93,san julian,MX,21.0167,-102.1667,10,
133,kavaratti,IN,10.5669,72.6420,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10_000
params = {
    'categories':'accommodation.hotel', 
    "apiKey":geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle: {lon}, {lat}, {radius}'
    params["bias"] = f'proximity: {lon}, {lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kedougou - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
lihue - nearest hotel: No hotel found
san julian - nearest hotel: No hotel found
kavaratti - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
vila do maio - nearest hotel: No hotel found
eyl - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
gaoual - nearest hotel: No hotel found
dharangaon - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
koundara - nearest hotel: No hotel found
yendi - nearest hotel: No hotel found
tshabong - nearest hotel: No hotel found
sandwip - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
zouerate - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
kapa'a - nearest hotel: No hotel found
freetown - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
bubaque - ne

,City,Country,Lat,Lng,Humidity,Hotel Name
13,kedougou,SN,12.5579,-12.1743,8,No hotel found
21,san luis de la loma,MX,17.2706,-100.8939,58,No hotel found
64,lihue,US,21.9789,-159.3672,64,No hotel found
93,san julian,MX,21.0167,-102.1667,10,No hotel found
133,kavaratti,IN,10.5669,72.6420,70,No hotel found
144,vila velha,BR,-20.3297,-40.2925,85,No hotel found
149,kidal,ML,18.4411,1.4078,10,No hotel found
155,vila do maio,CV,15.1333,-23.2167,82,No hotel found
186,eyl,SO,7.9803,49.8164,73,No hotel found
209,taoudenni,ML,22.6783,-3.9836,11,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE